In [180]:
import cv2
import os

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.video_visualizer import VideoVisualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.engine import DefaultTrainer
from detectron2.utils.visualizer import ColorMode
from detectron2.evaluation import COCOEvaluator

In [181]:
data_path = 'C:/Users/user/Desktop/RCP/ClimbAssistant/data'
output_dir = os.path.join(data_path, 'output_cosine')
filename = '7.jpg'

In [182]:
frame = cv2.imread(f'C:/Users/user/Desktop/RCP/ClimbAssistant/data/colour_check/{filename}') 

In [183]:
# dictionary to store BGR values for each colour
colour_values = {"YELLOW": (0,255,255), "GREEN": (0,255,0), "BLUE": (255,0,0), "RED": (0,0,255), "BLACK": (0,0,0), "WHITE": (255,255,255)}

configs = {'classes': 1}

In [184]:
# settle the model configs
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_1x.yaml"))
cfg.OUTPUT_DIR = output_dir
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = configs['classes']
predictor = DefaultPredictor(cfg)

In [185]:
# get outputs from the Mask RCNN model
outputs = predictor(frame)
# create a HSV frame to do colour categorization
hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

In [186]:
# Test colour on actual wall
# Output a image with HSV value of each hold for checking
def classifyHoldsTest(outputs, hsv_frame, colour_values):   
    colour_list = []
    contours = []

    output_frame = cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)

    for pred_mask in outputs['instances'].pred_masks:
        mask = pred_mask.cpu().numpy().astype('uint8') # extract mask from predictions

        # Extract the contour of each predicted mask and save it in a list
        contour, _ = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
        # get top of contour
        top = tuple(contour[0][contour[0][:, :, 1].argmin()][0])
        contours.append(contour[0])

        hsv_mean = cv2.mean(hsv_frame, mask=mask) # calculate mean HSV values of each mask
        hsv_mean = tuple([int(x) for x in hsv_mean])
    
        # get hue and value
        # hue is between 0 and 180, value is between 0 and 255
        hue = hsv_mean[0]
        saturation = hsv_mean[1]
        value = hsv_mean[2]

        # make use of hue to categorize colours
        color = "Undefined"
        if hue < 28:
            color = "YELLOW"
        elif hue < 78:
            color = "GREEN"
        elif hue < 120:
            color = "BLUE"
        else:
            color = "RED"
        
        # however, value is used to determine black and white, and hue is not useful
        # hence we reset the color if any of the holds are too dark or light as they should be classified as black and white respectively
        if value < 48:
            color = "BLACK"
        elif saturation < 65 and value > 133:
            color = "WHITE" 

        colour_list.append(color)
        cv2.putText(output_frame, str(hsv_mean), org=top, fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1.5, color=colour_values[color], 
        thickness=3)


    cv2.imwrite(os.path.join("C:/Users/user/Desktop/RCP/ClimbAssistant/data/colour_test", filename), output_frame)

In [187]:
classifyHoldsTest(outputs, hsv_frame, colour_values)